## Importing Dependencies

In [1]:
# Importing necessary tools
from flask import Flask
from flask_restful import Resource, Api, reqparse
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

In [ ]:
# Instantiating our DistilBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

## Building a processing function for predictions

In [ ]:
# Making a dictionary with class names for conversion
class_names = {0: "anger", 1: "joy", 2: "optimism", 3: "sadness"}

# A function containing the transformation steps from above
def logits_to_class_names(predictions):
    predictions = tf.nn.softmax(predictions.logits)
    predictions = tf.argmax(predictions, axis=1).numpy()
    predictions = [class_names[prediction] for prediction in predictions]
    
    return predictions

## Building the API

### Creating a Flask application

In [ ]:
# Setting up a Flask application
app = Flask(import_name=__name__)
api = Api(app=app)

In [ ]:
parser = reqparse.RequestParser()
parser.add_argument(name="Sequences", type=str, action="append",
                    help="The sequence to be classified", required=True)

### Building an endpoint for inference

In [ ]:
# Creating a class to represent our endpoint
class Inference(Resource):
    # A method corresponding to a GET request
    def get(self):
        # Parsing the arguments we defined earlier
        args = parser.parse_args()
        
        # Tokenizing the sequence
        sequence = tokenizer(args["Sequences"], return_tensors="tf")
        
        # Obtaining a prediction
        prediction = logits_to_class_names(model(sequence))
        
        # Returning the prediction
        return {"Predictions": prediction}, 200

In [ ]:
# Adding the endpoint to our app
api.add_resource(Inference, "/inference")

### Launching our application

In [ ]:
# launching our app
if __name__ == "__main__":
    app.run()